# Models ensemble by averaging

In general we saw that XGB overall is better (when sampling randomly train and test or in January and February tests), however, Random Forest error does not have a high variance and is better when predicting on March and April, thus we can do an average of the predicted number of sales of every Store in every day, leading to an overall result which is slightly better than the average of the 2 errors.

The other possiblity that we will evaluate is to do an average of the two predictions at a higher level, when we have already grouped by StoreID and Month, this could be even better if one of the model for instance tends to overestimate the predicted value and the over one underestimates it.

**NOTEBOOK GOAL**: Ensemble of models created by averaging the preditions

Averaged predictions:

- **1_RFR** - Notebook 5.3 Random forrest
- **2_XGB** - Notebook 6.4 XGBoost
- **3_AVG** - Notebook 7.0 AVG Monthly average


In [1]:
# reoder variables to work on train or test dataset
predict_for_submission = True
predict_for_submission = False

In [2]:
from import_man import *
import collections

from BIP import get_BIP_error, apply_BIP_submission_format

### Load predicted tests

**NOTE** If you cannot load the followig datasets, please go to the corresponding notebook and run it to generate the related dataset file. 

In [3]:
dfs_dict = collections.OrderedDict()
# the following dataset will be evaluated


months='m12'
if predict_for_submission:
    months='m34'

In [4]:
dfs_dict['RFR'] = pd.read_csv('./dataset/test_' + months + '_53_RFR_on_prep.csv')

In [5]:
dfs_dict['XGB'] = pd.read_csv('./dataset/test_' + months + '_64_Model_XGBoost_final.csv')

In [6]:
dfs_dict['AVG'] = pd.read_csv('./dataset/test_' + months + '_70_Model_monthly_average.csv')

FileNotFoundError: File b'./dataset/test_m12_70_Model_monthly_average.csv' does not exist

In [ ]:
# let's apply the apply_BIP_submission_format to all the dataframes
for mdl_lbl, df in dfs_dict.items():
    dfs_dict[mdl_lbl] = apply_BIP_submission_format(df)

We decided to work with the average of the already summed up value because it reduces more the error

In [ ]:
# make a copy of the first dataframe in order to use it as data structure
df_ens = list(dfs_dict.values())[0].copy()
df_ens['NumberOfSales'] = 0

In [ ]:
stores_to_spot = [1000, 1245, 1300, 1301] 

# Let's spot some random rows in order to prove that the average works correctly
for mdl_lbl, df in dfs_dict.items():
    print('................................ ' + mdl_lbl + '................................')
    print(df.loc[df.StoreID.isin(stores_to_spot)][['StoreID', 'Target', 'NumberOfSales']].head(40))


In [ ]:
df_ens[['StoreID', 'Target', 'NumberOfSales']].head()

In [ ]:
# sum up all
for mdl_lbl, df in dfs_dict.items():
    df_ens['NumberOfSales'] += df['NumberOfSales']

# divide by their number
df_ens['NumberOfSales'] /= len(dfs_dict)

df.loc[df.StoreID.isin(stores_to_spot)][['StoreID', 'Target', 'NumberOfSales']].head(40)

## Write to file

In [ ]:
if predict_for_submission:
    df_ens.to_csv('./dataset/submission_test_m34_82_Ensemble_average.csv', index=False)
else:
    df_ens.to_csv('./dataset/test_' + months + '_82_Ensemble_average.csv', index=False)